In [3]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, year, month, dayofmonth, current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# --- 1. CẤU HÌNH SPARK (Thêm Kafka JARs) ---
JAR_PATH = "/home/jovyan/jars"
jars = [
    f"{JAR_PATH}/hadoop-aws-3.3.4.jar",
    f"{JAR_PATH}/aws-java-sdk-bundle-1.12.262.jar",
    f"{JAR_PATH}/spark-sql-kafka-0-10_2.12-3.5.0.jar",       # <-- Mới
    f"{JAR_PATH}/kafka-clients-3.5.1.jar",                   # <-- Mới
    f"{JAR_PATH}/commons-pool2-2.11.1.jar",                  # <-- Mới
    f"{JAR_PATH}/spark-token-provider-kafka-0-10_2.12-3.5.0.jar" # <-- Mới
]

print("🚀 Khởi tạo Spark Streaming...")

spark = SparkSession.builder \
    .appName("Spotify Notebook Streaming") \
    .config("spark.jars", ",".join(jars)) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print("✅ Spark Session Ready!")

# --- 2. ĐỌC STREAM TỪ KAFKA ---
# Lưu ý: Dùng 'kafka:29092' vì đang chạy trong Docker Network
print("📡 Đang lắng nghe Kafka...")
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "playlist_events") \
    .option("startingOffsets", "latest") \
    .load()

# --- 3. XỬ LÝ DỮ LIỆU ---
event_schema = StructType([
    StructField("event_type", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("track_uri", StringType(), True),
    StructField("playlist_name", StringType(), True),
    StructField("timestamp", StringType(), True)
])

# Parse JSON từ cột 'value' của Kafka
parsed_df = kafka_df.select(from_json(col("value").cast("string"), event_schema).alias("data")) \
                    .select("data.*")

# Thêm cột thời gian để partition
output_df = parsed_df.withColumn("event_timestamp", col("timestamp").cast("timestamp")) \
                     .withColumn("year", year(col("event_timestamp"))) \
                     .withColumn("month", month(col("event_timestamp"))) \
                     .withColumn("day", dayofmonth(col("event_timestamp")))

# --- 4. GHI STREAM (Console hoặc MinIO) ---

# Cách 1: Ghi ra màn hình (Debug xem có dữ liệu không)
# query = output_df.writeStream \
#     .format("console") \
#     .outputMode("append") \
#     .start()

# Cách 2: Ghi vào MinIO (Production)
output_path = "s3a://spotify-streaming-data/"
checkpoint_path = "s3a://spotify-streaming-checkpoint/"

print(f"💾 Đang ghi dữ liệu vào: {output_path}")
query = output_df.writeStream \
    .format("parquet") \
    .outputMode("append") \
    .option("path", output_path) \
    .option("checkpointLocation", checkpoint_path) \
    .partitionBy("year", "month", "day") \
    .trigger(processingTime='10 seconds') \
    .start()

print("Streaming is running... (Nhấn Stop trên thanh công cụ để dừng)")
# Không dùng awaitTermination() trong Notebook vì nó sẽ treo Cell mãi mãi
# query.awaitTermination()

🚀 Khởi tạo Spark Streaming...
✅ Spark Session Ready!
📡 Đang lắng nghe Kafka...
💾 Đang ghi dữ liệu vào: s3a://spotify-streaming-data/
Streaming is running... (Nhấn Stop trên thanh công cụ để dừng)


In [4]:
print("🔍 KIỂM TRA KẾT QUẢ STREAMING...")

# Đọc thư mục Streaming vừa ghi
streaming_path = "s3a://spotify-streaming-data/"

try:
    df_result = spark.read.parquet(streaming_path)
    
    print(f"📊 Tổng số bản ghi đã nhận: {df_result.count()}")
    print("👀 5 dòng dữ liệu mới nhất:")
    
    # Sắp xếp theo thời gian giảm dần để xem cái mới nhất
    df_result.orderBy(col("event_timestamp").desc()).show(5, truncate=False)
    
except Exception as e:
    print("⚠️ Chưa có dữ liệu hoặc lỗi đọc (Có thể Spark chưa kịp commit file):")
    print(e)

🔍 KIỂM TRA KẾT QUẢ STREAMING...
📊 Tổng số bản ghi đã nhận: 72
👀 5 dòng dữ liệu mới nhất:
+----------+---+------------------+-------------+--------------------------+--------------------------+----+-----+---+
|event_type|pid|track_uri         |playlist_name|timestamp                 |event_timestamp           |year|month|day|
+----------+---+------------------+-------------+--------------------------+--------------------------+----+-----+---+
|play      |60 |spotify:track:5080|My Playlist 3|2025-12-09T15:05:12.791513|2025-12-09 15:05:12.791513|2025|12   |9  |
|like      |85 |spotify:track:9190|My Playlist 1|2025-12-09T15:05:10.790681|2025-12-09 15:05:10.790681|2025|12   |9  |
|play      |50 |spotify:track:2551|My Playlist 9|2025-12-09T15:05:08.789620|2025-12-09 15:05:08.78962 |2025|12   |9  |
|skip      |99 |spotify:track:8695|My Playlist 1|2025-12-09T15:05:06.786238|2025-12-09 15:05:06.786238|2025|12   |9  |
|play      |98 |spotify:track:7350|My Playlist 5|2025-12-09T15:05:04.785155|20